# Engine Insights

In [40]:
import os
import warnings

import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import plotly.io as pio
from pmlb import fetch_data
import seaborn as sns
import umap
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_feature_importances

pio.renderers.default = os.getenv("HOWSO_RECIPE_RENDERER", "notebook")

# Section 1: Load, Train, Analyze

### Step 1: Load Data

In [2]:
df = fetch_data('iris', local_cache_dir="../../data/iris")

df

,sepal-length,sepal-width,petal-length,petal-width,target
0,6.7,3.0,5.2,2.3,2
1,6.0,2.2,5.0,1.5,2
2,6.2,2.8,4.8,1.8,2
3,7.7,3.8,6.7,2.2,2
4,7.2,3.0,5.8,1.6,2
...,...,...,...,...,...
145,5.0,3.5,1.6,0.6,0
146,5.4,3.9,1.7,0.4,0
147,5.1,3.4,1.5,0.2,0
148,5.0,3.6,1.4,0.2,0


### Step 2: Train/Analyze

In [45]:
# Infer feature attributes
features = infer_feature_attributes(df)

# Create the Trainee
t = Trainee(
    features=features
)

# Train
t.train(df)

# Targeted Analysis
t.analyze()

# React into trainee
t.react_into_trainee(
    residuals = True
)

### Step 3: UMAP

In [35]:
# Get pairwise distances can be obtained with a single function call
dist_array = t.get_distances()['distances'].values

reducer = umap.UMAP(metric = 'precomputed')

# Ignores warnings about `inverse_transform`` not being available for precomputed distances
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    embedding = reducer.fit_transform(dist_array)


In [44]:
fig = px.scatter(x=embedding[:, 0], y=embedding[:, 1], title='UMAP')
fig.show()


### Step 4: Marginal Stats

Howso provides feature-level insight to the overall descriptive statistics of your dataset.

In [7]:
marginal_stats = t.get_marginal_stats()
marginal_stats

,petal-width,sepal-width,target,petal-length,sepal-length
kurtosis,-1.335246,0.241443,NaN,-1.395359,-0.573568
skew,-0.103944,0.330703,NaN,-0.271712,0.311753
max,2.500000,4.400000,NaN,6.900000,7.900000
variance,0.578532,0.186751,NaN,3.092425,0.681122
mean_absdev,0.658933,0.333093,NaN,1.561920,0.687556
uniques,22.000000,23.000000,3.000000,43.000000,35.000000
percentile_75,1.800000,3.300000,NaN,5.100000,6.400000
percentile_25,0.300000,2.800000,NaN,1.600000,5.100000
median,1.300000,3.000000,NaN,4.350000,5.800000
min,0.100000,2.000000,NaN,1.000000,4.300000


### Step 4: Which features are my data good at predicting?

*Interpretation*

Howso provides insights into whether a feature is predictable within a data analysis workflow. This can be analyzed depending on a feature's type, e.g., continuous vs. nomimal. 

For continuous features, Howso provides the feature's mean value and units and calculates its residual. The residuals are quantified by the mean absolute error (MAE) between a predicted value and actual value for a prediction and they describe the average amount a feature's predicted value varies from its real value. A feature with a small MAE is generally easier to predict than a feature with a large MAE. With this information, an SME will be able to determine the predictability of a feature, given the information contained in the remaining data. For additional context, Howso provides the R^2, RMSE, and Spearman coefficient for each feature. To compare between the continuous features, Howso orders the resulting features by highest R^2 and Spearman coefficient values, which are unitless and thus comparable between continuous features. This order represents a probable rank order of feature predictability. However, it is important that further analysis is performed to better understand these results, as they are dataset dependent.

For nominal features, Howso provides information on the feature's accuracy, precision, and recall. This provides a holistic view as to whether the feature can be predicted given the remaining information. To compare between the nominal feautres, Howso orders the nominal values by their F1 (possibly MCC) score, which is a unitless measurement that combines the accuracy, precision, and recall. This order represents a probable rank order of feature predictability. However, it is important that further analysis is performed to better understand these results, as they are dataset dependent.

Note, Engine insights provide a baseline set of information based on an analysis across all features, that points users in the correct direction of feature predictability. If an analysis is performed with a specified target feature, this analysis will become more precise, but the directionality will not change.

*Notes for Makana*

- Units: give option to fill in on IFA page; if you have date-time features, Howso should be able to automatically pull those units
- Ranking: in this notebook, I am ordering the dataframes by highest R^2 for continuous features and f1 score for nominal features, but we discussed presenting/bolding the "top" and "bottom" features in terms of predictability (with disclaimers). There are some additional questions relating to this, including how to determine the logic for the number of "good" features based on the number of features and how to show users all features and their predictability.
- Removing Features: there is an outstanding question as to how we can let users update the trainee if they want to try dropping features and see results, or just drop the features entirely. I'm not sure how we want to handle this.
- Nominal Features: TBD on if we will have an overall comparison in the MCC form. If necessary, we can calculate F1 (which is what I have done here), but Chris doesn't really like it.

In [8]:
pred_stats = t.get_prediction_stats(stats=['accuracy','mae','missing_value_accuracy','precision','r2','recall','rmse','spearman_coeff'])

In [9]:
print('Continuous Features')

print('The unit of measurement for each feature is centimeters.')

print('The mean, standard deviation, and prediction statistics for each feature are:')

u_eps = marginal_stats[['sepal_width','sepal_length','petal_width','petal_length']].loc[['mean','stddev']]

cts_stats = pred_stats[['sepal_width','sepal_length','petal_width','petal_length']].loc[['r2','spearman_coeff','mae','rmse',]]

cts_results = pd.concat([u_eps,cts_stats])

cts_results_T = cts_results.transpose()

cts_results_T.sort_values(by='r2',ascending=False)

Continuous Features
The unit of measurement for each feature is centimeters.
The mean, standard deviation, and prediction statistics for each feature are:


KeyError: "None of [Index(['sepal_width', 'sepal_length', 'petal_width', 'petal_length'], dtype='object')] are in the [columns]"

In [ ]:
print('Nominal Feature')

print('The F1 score, accuracy, precision, and recall of the nominal feature is:')

f1 = 2*pred_stats[['class']].loc['precision']*pred_stats[['class']].loc['recall']/(pred_stats[['class']].loc['precision']+pred_stats[['class']].loc['recall'])
print('F1:',f1['class'])

pred_stats[['class']].loc[['accuracy','precision','recall']]


### Step 5: Which features are important?

*Interpretation*

Feature importance is quantified by a combined analysis of how much each feature contributes to the prediction of another feature and whether that contribution improves or harms the accuracy of the prediction. For a given target (i.e., action, predicted) feature, Howso classifies each feature into one of four feature importance categories:

1. Red Flag Feature (RF): The feature contributes strongly to the prediction, but also adds uncertainty.

2. Important Feature (I): The feature contributes strongly to the prediction and reduces uncertainty.

3. Unimportant Feature (U): The feature contributes weakly to the prediction, but also adds uncertainty.

4. Tuning Feature (T): The feature contributes weakly to the prediction and reduces uncertainty.

Howso quantifies the contribution and uncertainty additions and reductions using each feature as the target feature in turn via following metrics:

Feature Contribution: the difference between a prediction of a target feature when each context feature is considered versus not considered during the prediction.

Mean Decrease in Accuracy (MDA): the mean decrease in accuracy of a target feature when each context feature is considered versus not considered during the prediction.

Howso summarizes the feature contribution and MDA results for each feature within matrices. To read a matrix, the label of each row (listed on the y-axis) indicates the target feature. Moving from left to right across each row of a matrix, the feature contribution or mda of each context feature for the target feature prediction can be identified. 

Note, the matrices also provide some insight into directional causality between features. This insight is useful for data scientists performing an exploratory data analysis (EDA).

*Notes for Makana*:

- Plot Accuracy Open Questions: Should feature contribution rows add up to one? the predicted feature value?
- Explanation: Need to develop explanatory language, collateral (video?) for how to interpret all these plots and the quantiles - needs a lot of work and understanding
- Plot Interactivity: Hover over boxes/rows/columns and get an explanation - can we add extra functionality? prompt user: what are you trying to predict? then provide insight directly around that prediction
- Advanced View: Could include MDA matrix, Difference plot
- Quantile Classification: What's the best way to do this? Right now I'm using the mean of the feature contribution values and mda for each feature, but how do we add in granualarity?


In [ ]:
contrib_matrix = t.get_contribution_matrix()

# Plot the heatmap
plt.figure(figsize=(10,8)) # You can adjust the size as needed
sns.heatmap(contrib_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
            xticklabels=contrib_matrix.columns.values, 
            yticklabels=contrib_matrix.columns.values)
plt.title('Feature Contribution Matrix Heatmap')
plt.show()

In [ ]:
mda_matrix = t.get_mda_matrix()

# Plot the heatmap
plt.figure(figsize=(10,8)) # You can adjust the size as needed
sns.heatmap(mda_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
            xticklabels=mda_matrix.columns.values, 
            yticklabels=mda_matrix.columns.values)
plt.title('MDA Matrix Heatmap')
plt.show()

In [ ]:
# Function to apply over the DataFrame
def check_quantile(row):
    # Exclude columns with value 1.0 and compute the mean of the remaining values
    filtered_row = row[row != 1.0]
    if filtered_row.empty:
        # If all values are 1.0, just return "small" for all to avoid errors
        return pd.Series(["small"] * len(row), index=row.index)
    mean = filtered_row.mean()
    # Compare each element to the mean and return "large", "small", or "skip"
    return row.apply(lambda x: "skip" if x == 1.0 else ("large" if x > mean else "small"))

# Apply the function along the axis=1 (i.e., row-wise)
contrib_matrix_cat = contrib_matrix.apply(check_quantile, axis=1)
mda_matrix_cat = mda_matrix.apply(check_quantile, axis=1)

In [ ]:
# Function to compare each element
def compare_elements(x, y):
    if x == 'skip' or y == 'skip':
        return 'skip'
    elif x == 'large' and y == 'large':
        return 'I'
    elif x == 'large' and y == 'small':
        return 'RF'
    elif x == 'small' and y == 'large':
        return 'RF'
    elif x == 'small' and y == 'small':
        return 'U'
    else:
        return 'T'

# Create a new dataframe by comparing df1 and df2 element-wise
comparison = pd.DataFrame(index=contrib_matrix_cat.index, columns=contrib_matrix_cat.columns)
for col in contrib_matrix_cat.columns:
    comparison[col] = np.vectorize(compare_elements)(contrib_matrix_cat[col], mda_matrix_cat[col])

print(comparison)

In [ ]:
def diagonal_difference_df(df):
    # Check if the DataFrame is square
    if df.shape[0] != df.shape[1]:
        raise ValueError("DataFrame must be square")
    
    # Compute the difference DataFrame
    difference_df = df - df.transpose()
    
    return difference_df

In [ ]:
diff_contrib_matrix = diagonal_difference_df(contrib_matrix)

# Plot the heatmap
plt.figure(figsize=(10,8)) # You can adjust the size as needed
sns.heatmap(diff_contrib_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
            xticklabels=diff_contrib_matrix.columns.values, 
            yticklabels=diff_contrib_matrix.columns.values)
plt.title('Contibution Matrix - Difference Heatmap')
plt.show()

### Step 6: Which cases are anomalous?

*Interpretation*

Howso provides insights into the anomalies within your dataset, utilizing the concept of "conviction" or how surprising a single data point is relative to the remaining data. The higher the surprisal (lower the conviction), the more likely the data point is anomalous. Howso can additionally classify anomalies as inliers vs. outliers, based on a data point's location in dataspace relative to other data. An outlier is an anomalous point that is very far away from any other data whereas an inlier is anomalous point that is very near (e.g., essentially overlapping) with another data point. As assessment of whether data points are outliers vs. inliers is an important tool for data preparation and results explanation within a data analytics workflow.

In [ ]:
t.react_into_features(
    distance_contribution = True, 
    familiarity_conviction_addition= True, 
    familiarity_conviction_removal= True, 
    influence_weight_entropy= True, 
    p_value_of_addition= True, 
    p_value_of_removal= True, 
    similarity_conviction= True)

conviction = t.get_cases(features=df.columns.to_list() +['distance_contribution','familiarity_conviction_addition','familiarity_conviction_removal','influence_weight_entropy','p_value_of_addition','p_value_of_removal','similarity_conviction'])

# Threshold to determine which cases will be deemed anomalous
conviction_threshold = 1.0

# Extract the anomalous cases
low_conviction = conviction[
    conviction['similarity_conviction'] <= conviction_threshold
].sort_values('similarity_conviction', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_conviction['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vice versa for inliers
cat = [
    'inlier' if d < average_dist_contribution else
    'outlier' for d in low_conviction['distance_contribution']
]
low_conviction['category'] = cat

print('There are',len(low_conviction),'datapoints with greater than average surprisal in the dataset. These cases are likely anomalous.')
print('There are',len(low_conviction.where(low_conviction['category']=='outlier').dropna()),'potential outliers in the dataset.')
print('There are',len(low_conviction.where(low_conviction['category']=='inlier').dropna()),'potential inliers in the dataset.')
print('The anomalous cases, in order of highest to lowest surprisal, are:')
low_conviction[['sepal_length','sepal_width','petal_length','petal_width','class','similarity_conviction','category']]